<a href="https://colab.research.google.com/github/MD-Ryhan/Text-Classification-with-Capsule-Network--Word2vec-glove/blob/main/word2vec%2BCapsule_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras import backend as K
from keras.layers import Activation
from keras.layers import Layer
from keras.layers import LeakyReLU, Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D , LSTM
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Model

2023-11-22 00:33:23.064431: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 00:33:23.065105: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 00:33:23.068882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 00:33:24.687293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional, Dense
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.metrics import Metric
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from tensorflow.keras import metrics

In [ ]:
# Load the training and test data from Excel files
train_data = pd.read_excel('eng-train.xlsx')
test_data = pd.read_excel('eng-test.xlsx')
val_data = pd.read_excel('eng-val.xlsx')

In [ ]:
# Split the data into features (X) and labels (y)
X_train = train_data['text'].tolist()
y_train = train_data['label'].tolist()
X_test = test_data['text'].tolist()
y_test = test_data['label'].tolist()
X_val = val_data['text'].tolist()
y_val = val_data['label'].tolist()

In [ ]:
# Word2Vec
# Train Word2Vec model on the training data
sentences = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences, vector_size=100, sg=1)

# Transform each text into a fixed-length vector using Word2Vec embeddings
X_train_w2v = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
                               or [np.zeros(100)], axis=0) for sentence in sentences])
X_test_w2v = np.array([np.mean([word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv]
                              or [np.zeros(100)], axis=0) for text in X_test])

In [ ]:
gru_len = 256
Routings = 3
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

max_features = 20000
maxlen = 100
embed_size = 256

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
# Pad the sequences to a fixed length
max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

In [ ]:
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [ ]:
training_padded = np.array(X_train)
training_labels = np.array(y_train)
val_padded = np.array(X_val)
val_labels = np.array(y_val)
testing_padded = np.array(X_test)
testing_labels = np.array(y_test)

In [ ]:
from tensorflow.keras.metrics import Metric

# Custom metric for F1 score
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(tf.math.round(y_pred), tf.float32)
        self.true_positives.assign_add(tf.reduce_sum(y_true * y_pred))
        self.false_positives.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.false_negatives.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
        return f1

In [ ]:
from keras import activations
from keras import backend as K
from keras.layers import Layer
import tensorflow

def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    return scale * x


#define our own softmax function instead of K.softmax
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)


#A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, share_weights=True, activation='squash', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        #final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:,:,:,0]) #shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            c = softmax(b, 1)
            # o = K.batch_dot(c, u_hat_vecs, [2, 2])
            o = tensorflow.einsum('bin,binj->bij', c, u_hat_vecs)
            if K.backend() == 'theano':
                o = K.sum(o, axis=1)
            if i < self.routings - 1:
                o = K.l2_normalize(o, -1)
                # b = K.batch_dot(o, u_hat_vecs, [2, 3])
                b = tf.einsum('bij,binj->bin', o, u_hat_vecs)
                if K.backend() == 'theano':
                    b = K.sum(b, axis=1)

        return self.activation(o)

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [ ]:
def get_model():
    input1 = Input(shape=(maxlen,))
    embed_layer = Embedding(vocab_size,
                            100,weights=[embedding_matrix],
                            input_length=maxlen)(input1)
    embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

    x = Bidirectional(GRU(gru_len,
                          activation='relu',
                          dropout=dropout_p,
                          recurrent_dropout=dropout_p,
                          return_sequences=True))(embed_layer)
    capsule = Capsule(
        num_capsule=Num_capsule,
        dim_capsule=Dim_capsule,
        routings=Routings,
        share_weights=True)(x)

    capsule = Flatten()(capsule)
    # capsule = Dropout(dropout_p)(capsule)
    # capsule = LeakyReLU()(capsule)
    capsule = Dense(128,activation='relu')(capsule)


    # x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(capsule)
    model = Model(inputs=input1, outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy',F1Score(),keras.metrics.TruePositives(), keras.metrics.TrueNegatives(),keras.metrics.FalsePositives(), metrics.FalseNegatives()])
    model.summary()

    return model

In [ ]:
model = get_model()

batch_size = 32
epochs = 5

2023-11-22 00:44:53.978852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1c:00.0, compute capability: 7.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          236000    
                                                                 
 spatial_dropout1d (Spatial  (None, 100, 100)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 100, 512)          549888    
 al)                                                             
                                                                 
 capsule (Capsule)           (None, 10, 16)            81920     
                                                                 
 flatten (Flatten)           (None, 160)               0     

In [ ]:
model.fit(training_padded, training_labels, batch_size=batch_size, epochs=epochs,
              validation_data=(val_padded, val_labels))

Epoch 1/5


2023-11-22 00:45:32.301610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-11-22 00:45:45.681302: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2ab180289fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 00:45:45.681413: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-11-22 00:45:46.019190: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 00:45:47.191821: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


30/30 [==============================] - 67s 1s/step - loss: 0.2683 - accuracy: 0.9280 - f1_score: 0.9626 - true_positives: 862.0000 - true_negatives: 1.0000 - false_positives: 57.0000 - false_negatives: 10.0000 - val_loss: 0.2352 - val_accuracy: 0.9376 - val_f1_score: 0.9678 - val_true_positives: 872.0000 - val_true_negatives: 0.0000e+00 - val_false_positives: 58.0000 - val_false_negatives: 0.0000e+00
Epoch 2/5
30/30 [==============================] - 26s 856ms/step - loss: 0.2346 - accuracy: 0.9376 - f1_score: 0.9678 - true_positives: 872.0000 - true_negatives: 0.0000e+00 - false_positives: 58.0000 - false_negatives: 0.0000e+00 - val_loss: 0.2314 - val_accuracy: 0.9376 - val_f1_score: 0.9678 - val_true_positives: 872.0000 - val_true_negatives: 0.0000e+00 - val_false_positives: 58.0000 - val_false_negatives: 0.0000e+00
Epoch 3/5
30/30 [==============================] - 27s 891ms/step - loss: 0.2404 - accuracy: 0.9376 - f1_score: 0.9678 - true_positives: 872.0000 - true_negatives: 0.00

In [ ]:
model.evaluate(testing_padded, testing_labels, verbose=2)

9/9 - 1s - loss: 0.2147 - accuracy: 0.9382 - f1_score: 0.9681 - true_positives: 243.0000 - true_negatives: 0.0000e+00 - false_positives: 16.0000 - false_negatives: 0.0000e+00 - 950ms/epoch - 106ms/step


[0.21467170119285583,
 0.9382239580154419,
 0.968127429485321,
 243.0,
 0.0,
 16.0,
 0.0]